In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

%config InteractiveShell.ast_node_interactivity = 'all'
#import dependencies

In [2]:
df = pd.read_parquet("C:/Users/Ariel/Desktop/tp taxi 2/Proyecto-Taxis-NYC/yellow_t_2018-01.parquet", engine="pyarrow")

Aplicamos un correcto formato de data a cada columna para su optimizacion e ingesta

In [3]:
def t_datime(df):
    df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
    df["hora"]= df.tpep_pickup_datetime.dt.floor("H")

    df.drop(columns=["congestion_surcharge","airport_fee","store_and_fwd_flag"], inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(round(df['total_amount'] ))
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float(df)
    return df

df =  total_transform(df)

Sacamos un dia del df para poder trabajarlo en esta etapa preeliminar

In [4]:
df_Impar = df[df.dayofmonth  == 3]

Filtro de fechas

In [5]:
def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

Nombramos los Borough con las locationsID

In [10]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("C:/Users/Ariel/Desktop/tp taxi 2/Proyecto-Taxis-NYC/taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

Creacion del dataframe de clima

In [13]:
#documentacion mismo nombre del clima en cada update

clima = pd.read_csv("C:/Users/Ariel/Desktop/tp taxi 2/Proyecto-Taxis-NYC/clima.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]
clima.datetime = clima['datetime'].apply(pd.to_datetime)

Transformacion de los datos para mejor optimizacion

In [14]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(round(clima['feelslike'] ))
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    return clima

clima = t_clima()

Se filtra el clima mes por mes para la ingesta

In [15]:
def control_clima(dat,clima):
    clima.sort_values(by='datetime', ascending=True, inplace=True)
    date = dat
    if (clima["datetime"] >= date).any():
        clima = clima[clima["datetime"] >= date]
        date_next= date + relativedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= relativedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1),clima)

Cambiamos el nombre de la columna para la conexion con la primary key

In [16]:
#clima in global

def aux ():
    clima.rename(columns={"datetime": "hora"},inplace=True)
    return clima

clima = aux()

In [11]:
#df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

Dropeamos valores menores a 0 y los ratecodeID 6 y 99

In [17]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==99].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount==0].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

Tratamiento de los outliers

In [18]:

def t_outliers(column):
    Q1 = df_Impar[column].quantile(0.01)
    Q3 = df_Impar[column].quantile(0.99)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliers_Sup_trip = (Q3 + (1.5*IQR)) 
    #se crea la columna marcador
    df_Impar["trip_distance_out"] = df_Impar.apply(lambda row: 1 if row[column]>outliers_Sup_trip else 0, axis=1)
    return df_Impar

def total_outliers():
    df_Impar=t_outliers("trip_distance")
    df_Impar=t_outliers("fare_amount")
    df_Impar=t_outliers("tip_amount")
    df_Impar=t_outliers("total_amount")
    return df_Impar

df_Impar=total_outliers()


df_Impar["outlier"] = df_Impar.apply(lambda row: 1 if row["trip_distance_out"]==1 or row["fare_amount_out"]==1 or
row["total_amount_out"]==1 or row["tip_amount_out"]==1 else 0, axis=1)


In [15]:
#df_Impar.to_parquet('',engine="pyarrow")

In [16]:
#df_Impar = pd.read_parquet("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet", engine="pyarrow")

Arrancamos con SQL

In [20]:
import mysql.connector as msql
from mysql.connector import Error
from sqlalchemy import create_engine
from multiprocessing import connection

In [18]:
#df total de los taxis
#df = pd.read_parquet("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet", engine="pyarrow")

In [21]:
df=df_Impar

In [19]:
#df = df[df.dayofmonth == 3]

In [24]:
#buscar los df ya tratados
df_Zona = pd.read_csv("C:/Users/Ariel/Desktop/tp taxi 2/Proyecto-Taxis-NYC/taxi+_zone_lookup.csv", sep=",")
#df_Clima=pd.read_csv("Proyecto-Taxis-NYC/clima.csv", sep=',')

Nos conectamos a la base de datos

In [25]:

try:
    conn = msql.connect(host='127.0.0.1', user='root',  
                        password='root')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS taxi_1;")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Database is created


In [26]:
host_name,db_name, u_name, u_pass, port_num  = "127.0.0.1","taxi_1", "root", "root", "3306"

In [27]:
'''try:
    connection = msql.connector.connect(
    host= host_name,
    user= u_name,
    password= u_pass,
    database= db_name
    )
  
    if connection.is_connected():
        print('Conexion exitosa')
        info_server = connection.get_server_info()
        print(info_server)
except Exception as ex:
    print(ex)'''

name 'mysql' is not defined


Creacion del motor de base de datos

In [28]:

engine = create_engine("mysql+mysqlconnector://" + u_name + ":" + u_pass + "@" 
                        + host_name + ":" + port_num + "/" + db_name, echo=False)

Importacion del dataframe a nuestra base de datos en SQL 

In [29]:

#CAMBIAR LOS NOMBRES DEL DF QUE QUEREMOS CARGAR Y EL NOMBRE QUE QUEREMOS QUE TENGA LA TABLA EN SQL
df.to_sql(name="taxis", con=engine, chunksize=200, if_exists="append", index=False)

-1327

In [30]:
df_Zona.to_sql(name="zona", con=engine, if_exists="append", index=False)

-1

In [31]:
clima.to_sql(name="clima", con=engine, if_exists="append", index=False)

-1